In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\3\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\3\test'
checkpoint_path = r'C:\Users\24306\Desktop\EfficientNetV2L\3\EfficientNetV2L.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[10.0, 80.0, 84.0, 100.0, 35.0, 80.0, 37.0, 52.0, 86.0, 13.0, 55.0, 84.0, 100.0, 93.0, 7.0, 52.0, 35.0, 45.0, 10.0, 95.0, 37.0, 47.0, 23.0, 4.0, 93.0, 95.0, 55.0, 47.0, 39.0, 52.0, 16.0, 40.0, 16.0, 37.0, 37.0, 47.0, 37.0, 55.0, 13.0, 30.0, 4.0, 7.0, 32.0, 13.0, 10.0, 7.0, 39.0, 16.0, 93.0, 47.0, 7.0, 35.0, 84.0, 93.0, 84.0, 37.0, 84.0, 30.0, 95.0, 86.0, 95.0, 30.0, 20.0, 93.0, 100.0, 37.0, 55.0, 95.0, 37.0, 80.0, 100.0, 66.0, 40.0, 52.0, 10.0, 7.0, 23.0, 7.0, 37.0, 37.0, 100.0, 20.0, 7.0, 37.0, 13.0, 23.0, 93.0, 13.0, 93.0, 80.0, 7.0, 37.0, 96.0, 95.0, 45.0, 13.0, 96.0, 86.0, 96.0, 45.0, 47.0, 16.0, 66.0, 37.0, 7.0, 47.0, 7.0, 40.0, 40.0, 96.0, 52.0, 96.0, 37.0, 4.0, 32.0, 13.0, 47.0, 100.0, 93.0, 84.0, 13.0, 37.0, 66.0, 23.0, 16.0, 45.0, 80.0, 80.0, 52.0, 10.0, 39.0, 30.0, 47.0, 86.0, 100.0, 86.0, 13.0, 93.0, 66.0, 93.0, 96.0, 32.0, 45.0, 10.0, 37.0, 84.0, 40.0, 100.0, 40.0, 37.0, 47.0, 84.0, 100.0, 40.0, 7.0, 7.0, 47.0, 10.0, 40.0, 37.0, 100.0, 45.0, 37.0, 84.0, 13.0, 66.0, 93.0, 37

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
#要改
EfficientNetV2L = tf.keras.applications.efficientnet_v2.EfficientNetV2L(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
EfficientNetV2L.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = EfficientNetV2L(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetv2-l (Functiona  (None, 7, 7, 1280)       117746848 
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 1281      
                                                                 
Total params: 117,748,129
Trainable params: 117,235,553
Non-trainable params: 512,576
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
#要改
np.savetxt('EfficientNetV2L_loss_3.txt',history.history['loss'])
#要改
np.savetxt('EfficientNetV2L_valloss_3.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Tue May  9 17:23:46 2023
Epoch 1/300
591/591 [==============================] - ETA: 0s - loss: 685.4741

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\3\EfficientNetV2L.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\3\EfficientNetV2L.ckpt\assets


591/591 [==============================] - 630s 994ms/step - loss: 685.4741 - val_loss: 148.3485
Epoch 2/300
591/591 [==============================] - ETA: 0s - loss: 68.2623

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\3\EfficientNetV2L.ckpt\assets


...

591/591 [==============================] - 1232s 2s/step - loss: 0.4572 - val_loss: 0.1286
Epoch 98/300
591/591 [==============================] - 1039s 2s/step - loss: 0.1835 - val_loss: 0.2669
Epoch 99/300
591/591 [==============================] - 1036s 2s/step - loss: 0.1706 - val_loss: 0.1828
Epoch 100/300
591/591 [==============================] - 1016s 2s/step - loss: 0.1522 - val_loss: 0.1728
Epoch 101/300
591/591 [==============================] - 1013s 2s/step - loss: 0.2992 - val_loss: 0.2291
Epoch 102/300
591/591 [==============================] - 1007s 2s/step - loss: 0.2013 - val_loss: 0.7605
Epoch 103/300
591/591 [==============================] - 1016s 2s/step - loss: 0.1540 - val_loss: 0.4230
Epoch 104/300
591/591 [==============================] - 1025s 2s/step - loss: 0.2377 - val_loss: 0.2982
Epoch 105/300
591/591 [==============================] - 1028s 2s/step - loss: 0.3141 - val_loss: 0.2316
Epoch 106/300
591/591 [==============================] - 1024s 2s/step 

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\3\EfficientNetV2L.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\3\EfficientNetV2L.ckpt\assets


591/591 [==============================] - 1317s 2s/step - loss: 0.1572 - val_loss: 0.1223
Epoch 142/300
591/591 [==============================] - 1113s 2s/step - loss: 0.2108 - val_loss: 0.2581
Epoch 143/300
591/591 [==============================] - 1109s 2s/step - loss: 0.1110 - val_loss: 0.2001
Epoch 144/300
591/591 [==============================] - 1109s 2s/step - loss: 0.1764 - val_loss: 0.2092
Epoch 145/300
591/591 [==============================] - 1118s 2s/step - loss: 0.1399 - val_loss: 0.2326
Epoch 146/300
591/591 [==============================] - 1122s 2s/step - loss: 0.1688 - val_loss: 0.2809
Epoch 147/300
591/591 [==============================] - 1116s 2s/step - loss: 0.1503 - val_loss: 0.2475
Epoch 148/300
591/591 [==============================] - 1102s 2s/step - loss: 0.2411 - val_loss: 0.2501
Epoch 149/300
591/591 [==============================] - 1091s 2s/step - loss: 0.1480 - val_loss: 0.2497
Epoch 150/300
591/591 [==============================] - 1092s 2s/ste

591/591 [==============================] - 1077s 2s/step - loss: 0.1006 - val_loss: 0.2525
Epoch 298/300
591/591 [==============================] - 1077s 2s/step - loss: 0.1325 - val_loss: 0.1797
Epoch 299/300
591/591 [==============================] - 1092s 2s/step - loss: 0.1024 - val_loss: 0.3411
Epoch 300/300
591/591 [==============================] - 1107s 2s/step - loss: 0.0697 - val_loss: 0.2167
end time: Sat May 13 04:26:49 2023
